<a href="https://colab.research.google.com/github/sergiocostaifes/PPCOMP_DM/blob/main/notebooks/07_baseline_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07_baseline_rf.ipynb — Baseline Supervisionado (Random Forest)

Objetivo  
Treinar e avaliar um baseline supervisionado com **Random Forest** sobre a base rotulada em janelas de 5 minutos (Notebook 06), comparando:

1. **Classificação binária**: `is_event = 1` para {BEFORE, DURING, AFTER} vs `NORMAL`
2. **Classificação multiclasse**: {NORMAL, BEFORE, DURING, AFTER}

A avaliação é feita com **split temporal** para evitar vazamento de informação futura.

Entradas (artefatos do pipeline)

- `window_5min_labeled.parquet` (Notebook 06)

Saídas (artefatos deste Notebook)

- `rf_binary.joblib`
- `rf_multiclass.joblib`
- `07_baseline_rf_summary.json`

Split temporal

Este notebook suporta duas estratégias:

- **Corte fixo** (default): treino = 80% inicial; teste = 20% final
- **TimeSeriesSplit** (opcional): validação em múltiplos folds temporais

Métricas reportadas

- Binário: accuracy, precision, recall, f1, ROC-AUC (quando aplicável)
- Multiclasse: accuracy, macro-f1, weighted-f1, classification report
- Matriz de confusão (binário e multiclasse)

Features

- Exclui colunas não-numéricas e colunas de rótulo (`state`, `is_critical`)
- Preserva `bucket_id` apenas como referência (não como feature)

Observações

- O dataset é desbalanceado, especialmente na classe DURING.
- Random Forest é utilizado como baseline interpretável e robusto.
- Ajustes como class_weight podem ser aplicados em iterações futuras.

In [1]:
# ============================================================
# 07_baseline_rf.ipynb
# Baseline supervisionado com Random Forest (binário vs multiclasse)
# Split temporal + métricas + matriz de confusão + salvamento do modelo
# ============================================================

# -----------------------------
# 0) BOOTSTRAP (Colab + Repo)
# -----------------------------
from pathlib import Path
import os
import sys
import subprocess
import importlib
import random
import numpy as np
import pandas as pd
import json

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Colab Drive
if not Path("/content/drive/MyDrive").exists():
    from google.colab import drive
    drive.mount("/content/drive")
else:
    print("[Bootstrap] Google Drive já montado.")

REPO_DIR = Path("/content/drive/MyDrive/Mestrado/PPCOMP_DM")
GITHUB_REPO = "https://github.com/sergiocostaifes/PPCOMP_DM.git"

if not REPO_DIR.exists():
    REPO_DIR.parent.mkdir(parents=True, exist_ok=True)
    print(f"[Bootstrap] Clonando repositório em: {REPO_DIR}")
    subprocess.run(["git", "clone", GITHUB_REPO, str(REPO_DIR)], check=True)
else:
    try:
        print("[Bootstrap] Atualizando repositório (git pull).")
        subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=True)
    except Exception as e:
        print("[Bootstrap] Aviso: não foi possível atualizar via git pull:", e)

os.chdir(str(REPO_DIR))
print("[Bootstrap] CWD =", os.getcwd())

repo_str = str(REPO_DIR)
if repo_str not in sys.path:
    sys.path.insert(0, repo_str)

importlib.invalidate_caches()

from src.paths import FEATURES_PATH, REPORTS_PATH, MODELS_PATH, ensure_dirs
ensure_dirs()

print("FEATURES_PATH =", FEATURES_PATH)
print("REPORTS_PATH  =", REPORTS_PATH)
print("MODELS_PATH   =", MODELS_PATH)

def log(msg: str) -> None:
    print(f"[07_baseline_rf] {msg}")

# -----------------------------
# 1) Imports ML
# -----------------------------
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    classification_report, confusion_matrix,
    roc_auc_score
)
import joblib

# -----------------------------
# 2) Carregar dataset rotulado
# -----------------------------
DATA_FILE = FEATURES_PATH / "window_5min_labeled.parquet"
assert DATA_FILE.exists(), f"Arquivo não encontrado: {DATA_FILE}"

df = pd.read_parquet(DATA_FILE).sort_values("bucket_id").reset_index(drop=True)
log(f"Dataset rotulado: shape={df.shape}")

assert "state" in df.columns, "Coluna 'state' ausente."
assert "bucket_id" in df.columns, "Coluna 'bucket_id' ausente."

# -----------------------------
# 3) Preparar features (X)
# -----------------------------
# Remover colunas de label / não numéricas
drop_cols = {"state"}  # alvo
# is_critical veio do NB05; pode existir e é derivado do NB04 — remover para não vazar definição
if "is_critical" in df.columns:
    drop_cols.add("is_critical")

# bucket_id: manter apenas para referência temporal, não como feature
drop_cols.add("bucket_id")

# Selecionar colunas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
X_cols = [c for c in numeric_cols if c not in drop_cols]

if len(X_cols) == 0:
    raise ValueError("Nenhuma feature numérica encontrada após filtragem.")

X = df[X_cols].copy()
y_multi = df["state"].astype(str).copy()

log(f"Features usadas: {len(X_cols)}")
log(f"Classes (multiclasse): {sorted(y_multi.unique().tolist())}")

# -----------------------------
# 4) Split temporal (corte fixo default)
# -----------------------------
SPLIT_MODE = "fixed"  # "fixed" ou "tscv"
TEST_RATIO = 0.20

n = len(df)
test_size = int(np.ceil(n * TEST_RATIO))
train_end = n - test_size

X_train, X_test = X.iloc[:train_end], X.iloc[train_end:]
y_train_multi, y_test_multi = y_multi.iloc[:train_end], y_multi.iloc[train_end:]

log(f"Split temporal: train={len(X_train)} test={len(X_test)} (modo={SPLIT_MODE})")

# -----------------------------
# 5) Cenário binário: evento vs normal
# -----------------------------
y_train_bin = (y_train_multi != "NORMAL").astype(int)
y_test_bin  = (y_test_multi  != "NORMAL").astype(int)

rf_bin = RandomForestClassifier(
    n_estimators=300,
    random_state=SEED,
    n_jobs=-1,
    class_weight="balanced_subsample"
)

rf_bin.fit(X_train, y_train_bin)
pred_bin = rf_bin.predict(X_test)
proba_bin = rf_bin.predict_proba(X_test)[:, 1] if hasattr(rf_bin, "predict_proba") else None

acc_bin = accuracy_score(y_test_bin, pred_bin)
prec_bin, rec_bin, f1_bin, _ = precision_recall_fscore_support(
    y_test_bin, pred_bin, average="binary", zero_division=0
)

auc_bin = None
if proba_bin is not None and len(np.unique(y_test_bin)) == 2:
    try:
        auc_bin = float(roc_auc_score(y_test_bin, proba_bin))
    except Exception:
        auc_bin = None

cm_bin = confusion_matrix(y_test_bin, pred_bin).tolist()

log(f"[BIN] acc={acc_bin:.4f} prec={prec_bin:.4f} rec={rec_bin:.4f} f1={f1_bin:.4f} auc={auc_bin}")

# -----------------------------
# 6) Cenário multiclasse: NORMAL/BEFORE/DURING/AFTER
# -----------------------------
rf_multi = RandomForestClassifier(
    n_estimators=400,
    random_state=SEED,
    n_jobs=-1,
    class_weight="balanced_subsample"
)

rf_multi.fit(X_train, y_train_multi)
pred_multi = rf_multi.predict(X_test)

acc_multi = accuracy_score(y_test_multi, pred_multi)
prec_m, rec_m, f1_m, _ = precision_recall_fscore_support(
    y_test_multi, pred_multi, average="macro", zero_division=0
)
prec_w, rec_w, f1_w, _ = precision_recall_fscore_support(
    y_test_multi, pred_multi, average="weighted", zero_division=0
)

labels = ["NORMAL", "BEFORE", "DURING", "AFTER"]
cm_multi = confusion_matrix(y_test_multi, pred_multi, labels=labels).tolist()
report_multi = classification_report(y_test_multi, pred_multi, labels=labels, zero_division=0)

log(f"[MULTI] acc={acc_multi:.4f} macro_f1={f1_m:.4f} weighted_f1={f1_w:.4f}")

# -----------------------------
# 7) (Opcional) TimeSeriesSplit rápido (apenas multiclasse)
# -----------------------------
tscv_results = []
if SPLIT_MODE == "tscv":
    tscv = TimeSeriesSplit(n_splits=5)
    for fold, (tr, te) in enumerate(tscv.split(X), start=1):
        X_tr, X_te = X.iloc[tr], X.iloc[te]
        y_tr, y_te = y_multi.iloc[tr], y_multi.iloc[te]
        clf = RandomForestClassifier(
            n_estimators=300,
            random_state=SEED,
            n_jobs=-1,
            class_weight="balanced_subsample"
        )
        clf.fit(X_tr, y_tr)
        y_hat = clf.predict(X_te)
        f1_macro = precision_recall_fscore_support(
            y_te, y_hat, average="macro", zero_division=0
        )[2]
        tscv_results.append({"fold": fold, "f1_macro": float(f1_macro)})

# -----------------------------
# 8) Salvar modelos
# -----------------------------
bin_model_path = MODELS_PATH / "rf_binary.joblib"
multi_model_path = MODELS_PATH / "rf_multiclass.joblib"

joblib.dump(rf_bin, bin_model_path)
joblib.dump(rf_multi, multi_model_path)

# -----------------------------
# 9) Summary (JSON) + prints úteis
# -----------------------------
summary = {
    "seed": SEED,
    "split_mode": SPLIT_MODE,
    "test_ratio": TEST_RATIO,
    "rows_total": int(n),
    "rows_train": int(len(X_train)),
    "rows_test": int(len(X_test)),
    "n_features": int(len(X_cols)),
    "features": X_cols,
    "class_distribution_total": df["state"].value_counts().to_dict(),
    "binary": {
        "acc": float(acc_bin),
        "precision": float(prec_bin),
        "recall": float(rec_bin),
        "f1": float(f1_bin),
        "roc_auc": auc_bin,
        "confusion_matrix": cm_bin
    },
    "multiclass": {
        "acc": float(acc_multi),
        "macro_f1": float(f1_m),
        "weighted_f1": float(f1_w),
        "confusion_matrix": cm_multi,
        "labels_order": labels,
        "classification_report_text": report_multi
    },
    "tscv_multiclass": tscv_results,
    "models": {
        "binary_path": str(bin_model_path),
        "multiclass_path": str(multi_model_path)
    }
}

summary_file = REPORTS_PATH / "07_baseline_rf_summary.json"
summary_file.write_text(json.dumps(summary, indent=2, ensure_ascii=False))

log("Notebook 07 finalizado com sucesso.")
print("\n=== MULTICLASS REPORT ===\n")
print(report_multi)
print("\n=== CONFUSION MATRIX (BIN) ===\n", cm_bin)
print("\n=== CONFUSION MATRIX (MULTI) labels=", labels, "===\n", cm_multi)
print("\nModels saved:\n", bin_model_path, "\n", multi_model_path)
print("\nSummary saved:\n", summary_file)

Mounted at /content/drive
[Bootstrap] Atualizando repositório (git pull).
[Bootstrap] CWD = /content/drive/MyDrive/Mestrado/PPCOMP_DM
FEATURES_PATH = /content/drive/MyDrive/Mestrado/02-datasets/03-features
REPORTS_PATH  = /content/drive/MyDrive/Mestrado/04-reports
MODELS_PATH   = /content/drive/MyDrive/Mestrado/03-models
[07_baseline_rf] Dataset rotulado: shape=(8914, 27)
[07_baseline_rf] Features usadas: 24
[07_baseline_rf] Classes (multiclasse): ['AFTER', 'BEFORE', 'DURING', 'NORMAL']
[07_baseline_rf] Split temporal: train=7131 test=1783 (modo=fixed)
[07_baseline_rf] [BIN] acc=0.8643 prec=0.9575 rec=0.5177 f1=0.6721 auc=0.8787599100887585
[07_baseline_rf] [MULTI] acc=0.8284 macro_f1=0.6554 weighted_f1=0.7730
[07_baseline_rf] Notebook 07 finalizado com sucesso.

=== MULTICLASS REPORT ===

              precision    recall  f1-score   support

      NORMAL       0.85      0.99      0.91      1304
      BEFORE       0.27      0.02      0.03       242
      DURING       1.00      1.00   

## Achados do Notebook 07 — Baseline Supervisionado (Random Forest)

### Estrutura experimental

- Total de janelas: 8914
- Split temporal (80/20) preservando ordem cronológica
- 24 features numéricas utilizadas
- Dois cenários avaliados:
  - Classificação binária (evento vs normal)
  - Classificação multiclasse (NORMAL / BEFORE / DURING / AFTER)

Não houve vazamento temporal.

---

## 1. Resultados — Classificação Binária

Métricas:

- Accuracy: 0.864
- Precision: 0.958
- Recall: 0.518
- F1-score: 0.672
- ROC-AUC: 0.879

Interpretação:

O modelo apresenta alta precisão e AUC robusta, indicando boa capacidade de separação entre janelas normais e janelas associadas a eventos.

O recall moderado sugere comportamento conservador, reduzindo falsos alarmes, característica desejável em ambientes críticos.

---

## 2. Resultados — Classificação Multiclasse

Métricas globais:

- Accuracy: 0.828
- Macro-F1: 0.655
- Weighted-F1: 0.773

Desempenho por classe:

- NORMAL: F1 ≈ 0.91
- BEFORE: F1 ≈ 0.03
- DURING: F1 = 1.00
- AFTER: F1 ≈ 0.68

---

## 3. Principais Achados

### 3.1 Excelente separação da classe DURING

A classe DURING foi identificada com precisão e recall perfeitos.

Isso indica que:

- As features estatísticas capturam fortemente a assinatura de degradação.
- A definição formal de episódios (Notebook 04) é consistente.
- A engenharia de atributos (Notebook 05) é adequada para detectar regimes críticos.

Este é um resultado estrutural importante para a dissertação.

---

### 3.2 Dificuldade em detectar BEFORE

A classe BEFORE apresentou recall extremamente baixo.

Isso sugere que:

- A transição pré-evento não apresenta separação estatística clara.
- Pode ser necessária modelagem sequencial.
- Pode ser interessante revisar a definição de janela BEFORE.
- Pode haver necessidade de features acumuladas ou temporais mais longas.

Esse comportamento é coerente com a natureza gradual de mudanças de regime.

---

### 3.3 AFTER apresenta regime intermediário

A classe AFTER mantém assinatura detectável, com F1 moderado.

Isso indica que:

- A estabilização pós-evento preserva traços estatísticos distintos.
- O modelo consegue distinguir recuperação parcial de normalidade plena.

---

## 4. Implicações Científicas

1. A segmentação temporal baseada em μ + 2σ mostrou-se operacionalmente válida.
2. Eventos críticos são estatisticamente separáveis.
3. A fase pré-evento requer modelagem temporal mais sofisticada.
4. O baseline Random Forest estabelece referência comparativa para modelos futuros.

---

## 5. Conclusão

O Notebook 07 confirma que a estrutura de dados construída ao longo dos Notebooks 01–06 é consistente e cientificamente utilizável.

A partir deste ponto, a pesquisa pode evoluir para:

- Modelos temporais mais avançados
- Ajuste de hiperparâmetros
- Técnicas de balanceamento
- Avaliação preditiva antecipada (forecasting de episódios)